In [1]:
import numpy as np
import csv
import string

x_train = np.genfromtxt('processed_train.txt', delimiter=',')
x_train = np.delete(x_train,-1,axis = 1)
#x_test = np.genfromtxt('processed_test.txt', delimiter=',')
#x_test = np.delete(x_test,-1,axis = 1)
with open('../data/train_labels.csv') as f:
    reader = csv.reader(f, delimiter=",")
    label_train = list(reader)   
del label_train[0]
label_train = np.asarray(label_train)[:,1]
label_train = label_train.tolist()

import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('../data/train_labels.csv')
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

lb_class = LabelEncoder()
obj_df['Category_code']=lb_class.fit_transform(obj_df['Category'])
obj_df[['Category','Category_code']]
y_train = np.asarray(obj_df)[:,1]
y_train = y_train.astype('int')

In [79]:
def softmax(a):
    result = []
    b = np.asarray(a)
    b = b.T
    b = b.tolist()
    for i in b:
        result.append(np.exp(i)/sum(np.exp(i)))
    result = np.asarray(result)
    result = result.T
    return result


def modelbuiler(n_layer, dim_layer, itera):
    x=[]
    x = np.concatenate((np.zeros((1,n_sample)),x_in.T),axis = 0)
    W = []
    W.append(np.random.randn(dim_layer+1,n_feature+1))
    if n_layer >=3:
        for i in range(1,n_layer-1):
            W.append(np.random.randn(dim_layer+1,dim_layer+1))
    W.append(np.random.randn(n_class,dim_layer+1))
    model = {}
    t = np.zeros((n_class,n_sample))
    for i in range(0,n_sample):
        t[y_in[i],i] = 1
    
    for iteration in range(0,itera):
        #Forward Prop
        a=[]
        z=[]
        a.append(np.dot(W[0],x))
        z.append(np.tanh(a[0]))
        z[0][0]=np.ones_like(z[0][0])
        if n_layer >=3:
            for i in range(1,n_layer-1):
                a.append(np.dot(W[i],z[i-1]))
                z.append(np.tanh(a[i]))
                z[i][0]=np.ones_like(z[i][0])
        a.append(np.dot(W[-1],z[-1]))
        z.append(softmax(a[-1]))
        y = z[-1]
        sum(sum(np.multiply(z[-1]-t,z[-1]-t)))
        print()
        
        #Back Prop Gradient Descent
        delta = []
        dW=[]
        h_prime = []
        for i in z:
            h_prime.append(np.ones_like(i)-np.multiply(i,i))
        delta.append(y-t)
        dW.append(np.dot(delta[0],z[1].T))
        if n_layer >= 3:
            for i in range(1,n_layer-1):
                delta.append(np.multiply(h_prime[-1-i],np.dot(W[-i].T,delta[-1])))
                dW.append(np.dot(delta[-1],z[-i-2].T))
        delta.append(np.multiply(h_prime[0],np.dot(W[1].T,delta[-1])))
        dW.append(np.dot(delta[-1],x.T))
        ddd = sum(sum(np.multiply(delta[1],delta[1])))
        print(ddd)
        for i in range(0,len(W)):
            W[i]+=-epsilon*dW[-i-1]
        
    model={'W':W}
    return model

#Estimate x
def guess(model, x_guess):
    W = model['W']
    x=[]
    x = np.concatenate((np.zeros((1,n_sample)),x_guess.T),axis = 0)
    a=[]
    z=[]
    a.append(np.dot(W[0],x))
    z.append(np.tanh(a[0]))
    z[0][0]=np.ones_like(z[0][0])
    if layer_number >=3:
        for i in range(1,layer_number-1):
            a.append(np.dot(W[i],z[i-1]))
            z.append(np.tanh(a[i]))
            z[i][0]=np.ones_like(z[i][0])
    a.append(np.dot(W[-1],z[-1]))
    z.append(softmax(a[-1]))
    y = z[-1]
    return y




In [16]:
x_in = x_train[0:5000]
y_in = y_train[0:5000]
x_try = x_train[5000:10000]
y_try = y_train[5000:10000]

n_sample = x_in.shape[0]
n_feature = x_in.shape[1]
n_class = 31

#Utilize NN Cross Validation
itera = 30
epsilon_valid = [0.0001]
layer_number_valid = [3]
layer_dimension_valid = [50]
error = np.zeros((len(epsilon_valid),len(layer_number_valid),len(layer_dimension_valid)))
for i,epsilon in enumerate(epsilon_valid):
    for j,layer_number in enumerate(layer_number_valid):
        for k,layer_dimension in enumerate(layer_dimension_valid):
            model = modelbuiler(layer_number,layer_dimension, itera)
            y_prob = guess(model, x_try)
            result = np.argmax(y_prob,0)
            diff = y_try-result
            diff = diff.astype(bool).astype(int)
            error[i,j,k] = sum(diff)
            print(error)
            
min_error = []
min_error_loc = []
for i in error:
    for j in i:
        min_error.append(min(j))
for i in np.where(error==min(min_error)):
    min_error_loc.append(i[0])
epsilon = epsilon_valid[min_error_loc[0]]
layer_number = layer_number_valid[min_error_loc[1]]
layer_dimension = layer_dimension_valid[min_error_loc[2]]


5002.549477312352
5003.320136582173
5001.886748927974
5000.7167304523
5001.123742935134
5000.371789206202
4999.585660993728
4999.054825365596
4999.203747740799
4998.546290590561
4998.415845500243
4998.234839700056
4998.453871185073
4998.397698930199
4998.071567693516
4998.061817485393
4997.892962200492
4997.634895929517
4997.630300067074
4997.601263252848
4997.905389386083
4998.107710035594
4998.044690410575
4998.037074156216
4998.023483623455
4998.014779757938
4998.007727156771
4998.0137973808005
4997.992001000078
4997.966511439087
[[[4703.]]]


In [80]:
from sklearn.metrics import classification_report

itera = 10000
epsilon = 0.00001
y_true = y_try
print(layer_number)
model = modelbuiler(layer_number,layer_dimension, itera)
y_prob = guess(model, x_try)
print(max(y_prob[0]))
result = np.argmax(y_prob,0)
print(classification_report(y_try, result))
print(result)

3

15052.841854323351

21693.551466011733

28523.022503783584

13284.604516943193

19468.405013721043

21693.44932725003

24732.779503860795

26147.854230888974

27476.23638283459

26287.362682723764

29937.115406336998

21273.544523593348

28906.029070871384

24622.937031831287

33061.18595818469

23745.866343957467

30124.41518679726

24797.147966660796

28127.806476089965

17432.54501794967

23850.265586601643

24156.865614493865

17239.00303795355

23030.025869273923

16988.26246140278

21527.058120342685

13088.052166170646

17852.711466482106

17624.40958874865

17326.658917832166

17472.16812069714

17546.68285299085

17691.876590474996

17875.274075223904

17503.47676587385

18298.238650927593

16408.645393730098

20472.94596357314

12264.166871222615

17953.49426247761

15394.058394806523

17633.79352449022

13539.883745440527

19148.005857775854

11080.43686786395

16270.330038683278

14709.010199900156

16148.353239788416

13858.850210376793

17266.927012610937

11030.493713


6335.2008382971035

6324.620910440239

6322.952279262129

6312.373919916398

6310.728173484553

6300.151470082794

6298.528449360656

6287.953544101663

6286.353042135679

6275.780127285233

6274.20189342369

6263.631206402795

6262.074950683806

6251.506768984737

6249.972166648087

6239.406802614578

6237.893498713932

6227.331294207889

6225.8389082919175

6215.280229272202

6213.808360115324

6203.253591146716

6201.801821507486

6191.251360226614

6189.819261609428

6179.273513169186

6177.8606505632615

6167.320022085976

6165.925958664676

6155.390853727147

6154.015155473687

6143.485968662377

6142.128208906787

6131.605320465847

6130.265084307859

6119.748854917224

6118.425743516382

6107.916509231343

6106.610143954104

6096.1082113367365

6094.818237743371

6084.323879223278

6083.049970899831

6072.563420391273

6071.305282619738

6060.826731438534

6059.584104719089

6049.113697829219

6047.886361267605

6037.424193901496

6036.211968482911

6025.758083176937

6024.560


4201.656335134226

4193.025444752393

4189.774971122247

4184.930030571429

4187.203124286923

4177.298849755082

4177.74663826824

4169.204764173912

4165.952144966901

4161.125834014046

4163.528433318527

4153.519799074066

4153.979081697366

4145.52486961502

4142.2739408296875

4137.462705660493

4139.987939966847

4129.882551234161

4130.35335614773

4121.985902345963

4118.738918168555

4113.940387961281

4116.576755369668

4106.3869967938

4106.868787000559

4098.5876025852485

4095.3461564265845

4090.5587550463124

4093.293764942903

4083.0332906049134

4083.5252214157254

4075.3300370252027

4072.095665823107

4067.318177284438

4070.139950893489

4059.8221115358074

4060.3232281536243

4052.2138480117096

4048.9880479423896

4044.219517287874

4047.116966168758

4036.7545197443606

4037.2638148300166

4029.2400774479975

4026.0244037493585

4021.2640458102233

4024.226947293216

4013.831966448524

4014.3484508599986

4006.410212626512

4003.206237525809

3998.4534556405556


2839.9750584329736

2833.1631544071347

2833.6621492725585

2828.864567156021

2831.5019102256574

2823.749488954714

2824.01015006126

2819.3874055217643

2820.222586500028

2814.1255600023424

2814.969218992723

2809.9732124803704

2812.540632207948

2805.1699038144257

2805.4843427406645

2800.79024454584

2802.613211449604

2795.6888314272173

2796.1301074143303

2791.426934441899

2793.8660835482556

2786.4864782509017

2786.8072428448477

2782.2055620629276

2783.813201120738

2777.225169449643

2777.787850693094

2773.0726747148638

2775.7948090732975

2768.3249718749485

2768.617255525375

2764.074165529721

2765.5001435271324

2759.1769582167753

2759.8677869296366

2755.0872004034336

2757.8388918738865

2750.503063714319

2750.8102532208804

2746.271407288607

2747.9663300368916

2741.497927028003

2742.102812112332

2737.4000364946314

2740.1228505376935

2732.87753664484

2733.200484944455

2728.692081470283

2730.49277270515

2724.0429119464643

2724.6553070593504

2719.


2173.8683446788577

2172.8934076782534

2171.920713857966

2170.9503497348123

2169.982244803313

2169.0164522402047

2168.0529284428553

2167.0917053157036

2166.1327563240557

2165.176100066413

2164.2217206393902

2163.269628497515

2162.3198142345

2161.3722834400032

2160.4270306008048

2159.484058482077

2158.5433638563945

2157.604947904748

2156.668808714839

2155.7349466181786

2154.803360444189

2153.874050096877

2152.9470148160904

2152.022254311314

2151.0997680451806

2150.179555653686

2149.2616167180267

2148.345950857442

2147.432557712292

2146.5214369063224

2145.612588103456

2144.7060109330005

2143.801705059015

2142.899670105838

2141.999905718325

2141.1024115001564

2140.207187057718

2139.3142319541485

2138.4235457370455

2137.5351279062447

2136.648977928458

2135.7650952140543

2134.883479123462

2134.004128949712

2133.1270439183054

2132.252223173681

2131.3796657746993

2130.5093706828357

2129.64133675504

2128.775562732297

2127.9120472305462

2127.05

...


1178.6626218011663

1163.9267759400327

1178.2770166124506

1163.6400585736842

1177.9191806016602

1163.3757045520356

1177.5806688515295

1163.1283102401217

1177.2572420468566

1162.8951491429502

1176.9464525751027

1162.6746175925932

1176.646627162662

1162.4656303115974

1176.3564601405806

1162.2673795204103

1176.0748419640363

1162.0792346135822

1175.8007785997565

1161.90069745994

1175.5333460321826

1161.7313808868169

1175.271657890649

1161.5709979648127

1175.014836995728

1161.4193574504854

1174.7619863994503

1161.2763638101326

1174.5121572096798

1161.1420215467945

1174.2643111225661

1161.0164441698162

1174.0172761225285

1160.8998684450326

1173.769695159677

1160.7926745304173

1173.519971360053

1160.695411846947

1173.2662227350502

1160.6088279807188

1173.0062811230823

1160.5338909443333

1172.7378164994332

1160.4717778126649

1172.4587560705

1160.4237628816636

1172.1683012258989

1160.3908567580243

1171.868933439676

1160.3729210258196

1171.569458


1164.5291260524339

1152.3485030786144

1164.5358696486476

1152.3532051547877

1164.543233085435

1152.358489871658

1164.5512267548247

1152.3643676370075

1164.5598615702509

1152.370849397128

1164.5691489944816

1152.3779466652888

1164.579101068726

1152.3856715519673

1164.5897304437915

1152.394036796458

1164.6010504129413

1152.403055801088

1164.613074946729

1152.4127426663956

1164.6258187298533

1152.4231122289975

1164.6392971999687

1152.4341801011872

1164.6535265887906

1152.445962712813

1164.668523965476

1152.4584773551

1164.6843072819383

1152.4717422271829

1164.700895421174

1152.4857764844462

1164.7183082471643

1152.5006002895213

1164.7365666576611

1152.5162348654173

1164.7556926395396

1152.5327025510396

1164.7757093257662

1152.5500268588503

1164.7966410552544

1152.568232534585

1164.8185134345863

1152.5873456191446

1164.841353401603

1152.6073935119673

1164.8651892906603

1152.6284050356571

1164.8900508992795

1152.6504105022523

1164.915969555


1193.0196987128497

1179.1082007062912

1193.2132164373068

1179.2825496780622

1193.4065047052618

1179.4564337780962

1193.5995590234447

1179.6298431163284

1193.792374903199

1179.8027676654594

1193.9849478524866

1179.9751972511683

1194.1772733679138

1180.147121543511

1194.3693469260681

1180.3185300476448

1194.5611639746387

1180.4894120949805

1194.7527199230124

1180.6597568345278

1194.9440101324137

1180.8295532245754

1195.1350299055327

1180.9987900245428

1195.3257744755813

1181.1674557875956

1195.5162389947552

1181.3355388534328

1195.7064185218394

1181.5030273419773

1195.8963080094522

1181.6699091477633

1196.0859022902118

1181.836171935381

1196.275196062232

1182.0018031358716

1196.4641838734378

1182.1667899444435

1196.652860105313

1182.331119319902

1196.8412189553856

1182.4947779854945

1197.0292544184724

1182.6577524320733

1197.2169602670106

1182.8200289233723

1197.4043300300123

1182.9815935040617

1197.591356970795

1183.142432010754

1197.77


1220.077748843214

1231.478947820224

1220.3203691898987

1231.7040584239032

1220.56269029276

1231.9290876292562

1220.8047149298916

1232.1540324840057

1221.0464454090213

1232.3788960143559

1221.2878853050315

1232.6036735371601

1221.5290370071166

1232.8283721248663

1221.7699052094954

1233.0529829088848

1222.010491966559

1233.2775205373077

1222.2508037317898

1233.501966934456

1222.4908413621213

1233.7263515337597

1222.7306143524665

1233.9506367738727

1222.9701205903918

1234.1748816087807

1223.2093751959687

1234.3990079600997

1223.4483694659214

1234.6231344102969

1223.6871293662891

1234.8470993784208

1223.9256322326085

1235.0711425292031

1224.1639254299741

1235.2949310768204

1224.401957387428

1235.5189509679599

1224.639818173228

1235.7425197148602

1224.877397115087

1235.9666239388082

1225.1148698846898

1236.1898693033158

1225.3520059470106

1236.4142582445497

1225.5891526603225

1236.6369526456233

1225.8258379039396

1236.8620095950528

1226.062

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [65]:
a=[np.array([1,2]), np.array([3,4])]
b = np.asarray(a)
b=b.T
print(a)
print(b.tolist())

[array([1, 2]), array([3, 4])]
[[1, 3], [2, 4]]
